In [ ]:
import os
import pandas as pd
import torch
from neuralprophet import NeuralProphet, set_log_level

# Configuração Inicial
device = "cuda" if torch.cuda.is_available() else "cpu"  # Define o dispositivo (GPU ou CPU)

# Carregar os Dados
data_path = "new_stats.csv"  # Caminho para o dataset


In [ ]:
df = pd.read_csv(data_path)

# Renomear as Colunas
df.rename(columns={time_col: "ds", target_col: "y"}, inplace=True)
df["ds"] = pd.to_datetime(df["ds"])

last_date = df['ds'].iloc[-1]
dates = pd.date_range(start=last_date + pd.Timedelta(minutes=1), periods=predict_size, freq='T')
future_dates = pd.DataFrame({'ds': dates})
future_dates['open'] = df['open'].iloc[-1]
future_dates['tick_volume'] = df['tick_volume'].iloc[-1]
future_dates['real_volume'] = df['real_volume'].iloc[-1]
future_dates['y'] = None

# print(future_dates)

df_future = pd.concat([df, future_dates], ignore_index=True)


In [ ]:
model = NeuralProphet()

model.add_future_regressor('open')
model.add_future_regressor('tick_volume')
model.add_future_regressor('real_volume')


# Define o dispositivo (CPU ou GPU)
if torch.cuda.is_available():
    model.device =  "cuda"
else:
    model.device = "cpu"

print(f"model.device: {model.device}")

set_log_level("ERROR")
# set the model to expect these events
#m = m.add_events(["playoff", "superbowl"])
# create the data df with events
# history_df = model.create_df_with_events(df, )
metrics = model.fit(df, freq="1min")


forecast = model.predict(df=df_future)



In [ ]:
forecast.to_csv("IGOR123123.CSV")

# Visualize the forecast
model.plot(forecast)